# Baseline Verification

Verify the pre-optimized baseline submission score.

In [ ]:
import numpy as np
import pandas as pd

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

def score_group(xs, ys, degs):
    """Calculate score for a single N-tree configuration"""
    n = len(xs)
    all_x, all_y = [], []
    for i in range(n):
        rad = np.radians(degs[i])
        c, s = np.cos(rad), np.sin(rad)
        for j in range(len(TX)):
            x = TX[j] * c - TY[j] * s + xs[i]
            y = TX[j] * s + TY[j] * c + ys[i]
            all_x.append(x)
            all_y.append(y)
    side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
    return side * side / n

def calculate_total_score(df):
    """Calculate total score for a submission"""
    # Parse the 's' prefix from values
    df['x_val'] = df['x'].str[1:].astype(float)
    df['y_val'] = df['y'].str[1:].astype(float)
    df['deg_val'] = df['deg'].str[1:].astype(float)
    
    # Extract N from id (e.g., '003_1' -> 3)
    df['n'] = df['id'].str.split('_').str[0].astype(int)
    
    total_score = 0
    per_n_scores = {}
    
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) == n:
            xs = group['x_val'].values
            ys = group['y_val'].values
            degs = group['deg_val'].values
            score = score_group(xs, ys, degs)
            per_n_scores[n] = score
            total_score += score
        else:
            print(f"Warning: N={n} has {len(group)} trees instead of {n}")
    
    return total_score, per_n_scores

print("Loading baseline submission...")
df = pd.read_csv('baseline.csv')
print(f"Shape: {df.shape}")
print(df.head())

In [ ]:
# Calculate total score
total_score, per_n_scores = calculate_total_score(df)
print(f"\nTotal Score: {total_score:.6f}")
print(f"\nScore breakdown by N range:")
print(f"  N=1-50:   {sum(per_n_scores[n] for n in range(1, 51)):.4f}")
print(f"  N=51-100: {sum(per_n_scores[n] for n in range(51, 101)):.4f}")
print(f"  N=101-150: {sum(per_n_scores[n] for n in range(101, 151)):.4f}")
print(f"  N=151-200: {sum(per_n_scores[n] for n in range(151, 201)):.4f}")

In [ ]:
# Show worst N values (highest per-N scores)
print("\nTop 20 worst N values (highest per-N scores):")
sorted_scores = sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    print(f"  N={n}: {score:.6f}")

In [ ]:
# Copy to submission folder
import shutil
shutil.copy('baseline.csv', '/home/submission/submission.csv')
print("Copied baseline to /home/submission/submission.csv")

In [ ]:
# Save metrics
import json
metrics = {'cv_score': total_score}
with open('metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")